## Building Database Agent for query and visualize the data 

In [34]:
%pip install smolagents[telemetry]

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: smolagents[telemetry] in c:\users\janit\anaconda3\envs\torchenv\lib\site-packages (1.11.0)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 578.7 kB/s eta 0:00:06
   ----- ---------------------------------- 0.5/3.8 MB 578.7 kB/s eta 0:00:06
   -------- ------------------------------- 0.8/3.8 MB 589.1 kB/s eta 0:00:06
   -------- ------------------------------- 0.8/3.8 MB 589.1 kB/s eta 0:00:06
   ---------- ----------------------------- 1.0/3.8 MB 621.7 kB/s eta 0:00:05
   ---------- -

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-openai 0.3.1 requires openai>=1.1.0, but you have openai 0.28.0 which is incompatible.
llama-index-legacy 0.9.48.post4 requires openai>=1.1.0, but you have openai 0.28.0 which is incompatible.
llama-index-llms-openai 0.3.19 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.


In [2]:
! pip install langfuse

  Using cached langfuse-2.60.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Using cached langfuse-2.60.2-py3-none-any.whl (274 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)


### we can set up the a tracer-provider for our configured OpenTelemetry.

### Add traces to the lang-fuse

In [1]:
import os
import base64

# Get your own keys from https://cloud.langfuse.com
LANGFUSE_PUBLIC_KEY = "pk-lf-8c56b4f9-6df5-4dc1-bbd1-d42a65ccdf1f"
LANGFUSE_SECRET_KEY = "sk-lf-a2f61880-91b4-4549-b856-cf5b1b72ddc5"
os.environ["LANGFUSE_PUBLIC_KEY"] = LANGFUSE_PUBLIC_KEY
os.environ["LANGFUSE_SECRET_KEY"] = LANGFUSE_SECRET_KEY
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 EU region example
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 US region example

LANGFUSE_AUTH = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [2]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
 
# Create a TracerProvider for OpenTelemetry
trace_provider = TracerProvider()

# Add a SimpleSpanProcessor with the OTLPSpanExporter to send traces
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# Set the global default tracer provider
from opentelemetry import trace
trace.set_tracer_provider(trace_provider)
tracer = trace.get_tracer(__name__)

# Instrument smolagents with the configured provider
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [4]:
from langfuse.decorators import observe

In [5]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-a2f61880-91b4-4549-b856-cf5b1b72ddc5",
  public_key="pk-lf-8c56b4f9-6df5-4dc1-bbd1-d42a65ccdf1f",
  host="https://us.cloud.langfuse.com"
)

In [10]:
from smolagents import HfApiModel, CodeAgent
from smolagents import HfApiModel, LiteLLMModel, OpenAIServerModel, TransformersModel, tool
from smolagents.agents import CodeAgent, ToolCallingAgent


In [7]:
from smolagents import HfApiModel, CodeAgent
from smolagents import HfApiModel, LiteLLMModel, OpenAIServerModel, TransformersModel, tool
from smolagents.agents import CodeAgent, ToolCallingAgent

model = LiteLLMModel(
        model_id="ollama/qwen2.5-coder:1.5b",
        api_base="http://localhost:11434",  # replace with remote open-ai compatible server if necessary
        api_key="your-api-key",  # replace with API key if necessary
        num_ctx=8192,  # ollama default is 2048 which will often fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
    )

# Create a simple agent to test instrumentation
@observe()
def complete():
    agent = CodeAgent(
        tools=[],
        model=model
    )

    agent.run("1+1=")

In [8]:
@observe()
def main():
    return complete()

In [9]:
main()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1+1=                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:1.5b ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  sum_result = 1 + 1                                                                                               
  final_answer(sum_result)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 24.22 seconds| Input tokens: 2,035 | Output tokens: 44]

In [10]:
from smolagents import CodeAgent, tool, HfApiModel

# Tool to suggest a menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """
    tool that Suggest menu for a party.
    Suggests a menu based on the occasion.
    You can only pass the values one of them ("casual","formal","superhero","custom")
    Args:
        occasion (str): The type of occasion for the party.Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

# Alfred, the butler, preparing the menu for the party
agent = ToolCallingAgent(tools=[suggest_menu], model=model,max_steps=5)

# Preparing the menu for the party
agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:1.5b ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'suggest_menu' with arguments: {'occasion': 'formal'}                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 3-course dinner with wine and dessert.

[Step 1: Duration 10.52 seconds| Input tokens: 1,254 | Output tokens: 22]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "We will serve an elegant formal menu featuring gourmet │
│ dishes, a rich selection of wine pairings, and a thoughtful beverage list. We'll ensure your guests have a      │
│ memorable experience."}                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: We will serve an elegant formal menu featuring gourmet dishes, a rich selection of wine pairings, and
a thoughtful beverage list. We'll ensure your guests have a memorable experience.

[Step 2: Duration 2.71 seconds| Input tokens: 2,584 | Output tokens: 80]

"We will serve an elegant formal menu featuring gourmet dishes, a rich selection of wine pairings, and a thoughtful beverage list. We'll ensure your guests have a memorable experience."

### Add traces to the to the phoenix 

In [2]:
import os
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = "api_key=a2b620825c6237cfc7b:445bc25";
os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=a2b620825c6237cfc7b:445bc25";
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com";

In [11]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-llm-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


OpenTelemetry Tracing Details
|  Phoenix Project: my-llm-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [12]:

model = LiteLLMModel(
        model_id="ollama/qwen2.5-coder:1.5b",
        api_base="http://localhost:11434",  # replace with remote open-ai compatible server if necessary
        api_key="your-api-key",  # replace with API key if necessary
        num_ctx=8192,  # ollama default is 2048 which will often fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
    )

In [13]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool
from smolagents.agents import CodeAgent, ToolCallingAgent

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = ToolCallingAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ], 
    model=model,
    max_steps=5,
    verbosity_level=2
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme    │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:1.5b ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
ModelResponse(id='chatcmpl-f7d992ef-46a6-4957-83ce-448bf7769532', created=1743662481,                              
model='ollama/qwen2.5-coder:1.5b', object='chat.completion', system_fingerprint=None,                              
choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant',              
tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"answer": "You can create a villain-themed 
party by dressing up as iconic villains, like Mr. Freeze, Mr. Freeze II, or Mr. Freeze III. You can also choose    
costumes that feature famous superheroes like Superman, Batman, Spider-Man, Iron Man, Black Widow, Captain America,
Thor, Hulk, Ant-Man, and Hawkeye. Make sure to wear costumes in shades of black to match the theme. Another idea is
to create a costume party where each person wears a costume and then has a mini party with their costume           
friend."}', name='final_answer'), id='call_b5ceabac-4b92-4220-8372-0daeefe6837c', type='function')],               
function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=121, prompt_tokens=1750,       
total_tokens=1871, completion_tokens_details=None, prompt_tokens_details=None))                                    

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'You can create a villain-themed party by dressing up   │
│ as iconic villains, like Mr. Freeze, Mr. Freeze II, or Mr. Freeze III. You can also choose costumes that        │
│ feature famous superheroes like Superman, Batman, Spider-Man, Iron Man, Black Widow, Captain America, Thor,     │
│ Hulk, Ant-Man, and Hawkeye. Make sure to wear costumes in shades of black to match the theme. Another idea is   │
│ to create a costume party where each person wears a costume and then has a mini party with their costume        │
│ friend.'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: You can create a villain-themed party by dressing up as iconic villains, like Mr. Freeze, Mr. Freeze 
II, or Mr. Freeze III. You can also choose costumes that feature famous superheroes like Superman, Batman, 
Spider-Man, Iron Man, Black Widow, Captain America, Thor, Hulk, Ant-Man, and Hawkeye. Make sure to wear costumes in
shades of black to match the theme. Another idea is to create a costume party where each person wears a costume and
then has a mini party with their costume friend.

[Step 1: Duration 19.79 seconds| Input tokens: 1,750 | Output tokens: 121]

'You can create a villain-themed party by dressing up as iconic villains, like Mr. Freeze, Mr. Freeze II, or Mr. Freeze III. You can also choose costumes that feature famous superheroes like Superman, Batman, Spider-Man, Iron Man, Black Widow, Captain America, Thor, Hulk, Ant-Man, and Hawkeye. Make sure to wear costumes in shades of black to match the theme. Another idea is to create a costume party where each person wears a costume and then has a mini party with their costume friend.'

In [27]:
agent.run("catering service in Gotham City")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ catering service in Gotham City                                                                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:1.5b ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Exception while exporting Span.
Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
ModelResponse(id='chatcmpl-562897ea-0432-4427-8e5e-228a8dc1486f', created=1743610491,                              
model='ollama/qwen2.5-coder:1.5b', object='chat.completion', system_fingerprint=None,                              
choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant',              
tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"query": "a search term for finding        
catering services."}', name='catering_service_tool'), id='call_4ed77150-63b4-4d1f-a307-8ddc871308ec',              
type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=25,         
prompt_tokens=1729, total_tokens=1754, completion_tokens_details=None, prompt_tokens_details=None))                

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'catering_service_tool' with arguments: {'query': 'a search term for finding catering services.'} │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Final answer: The highest-rated catering service in Gotham City is Goody's Catering.

Exception while exporting Span.
Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib

[Step 2: Duration 13.82 seconds| Input tokens: 3,540 | Output tokens: 56]

Exception while exporting Span.
Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janit\anaconda3\envs\torchenv\Lib

"The highest-rated catering service in Gotham City is Goody's Catering."

In [7]:
# import relevant libraries for building agent
import pandas as pd
import duckdb
from pprint import pprint
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool
from smolagents.agents import CodeAgent, ToolCallingAgent
from smolagents.tools import tool


In [28]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
 
# Create a TracerProvider for OpenTelemetry
trace_provider = TracerProvider()

# Add a SimpleSpanProcessor with the OTLPSpanExporter to send traces
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# Set the global default tracer provider
from opentelemetry import trace
trace.set_tracer_provider(trace_provider)
tracer = trace.get_tracer(__name__)

# Instrument smolagents with the configured provider
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


In [29]:
agent = CodeAgent(
        tools=[],
        model=model
    )

agent.run("1+1=")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1+1=                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:1.5b ─────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1                                                                                                   
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 3.23 seconds| Input tokens: 2,013 | Output tokens: 35]

2

In [8]:
# define the path to the transactional data
TRANSACTION_DATA_FILE_PATH = 'D:/AI-Agent-for-Healthcare-Appointment-Booking/test/agent_sample/Store_Sales_Price_Elasticity_Promotions_Data.parquet'

In [9]:
# prompt template for tool 1
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

Code-First Approach: First-class support for Code Agents that write their actions directly in code, removing the need for parsing and simplifying tool calling.

### 1. There is a tool for generating sql query from given text 

In [8]:
# create the code agent with the help of the code-agent from hugging face smolagents
llm = HfApiModel()
agent = CodeAgent(tools=[generate_sql_query,execute_query_tool], 
                  model=llm
                  )

In [9]:
query_agent = CodeAgent(tools=[],
                        model=llm)

In [10]:
test_prompt = "generate sql query and execute to show all the sales for store 1320 on November 1st, 2021"

In [11]:
@tool
def generate_sql_query(prompt: str) -> str:
    """
    Generates an SQL query based on the user prompt and the given table schema.
    prompt is simple string you can only pass sting value

    Args:
        prompt (str): The user's request in natural language. This is only input to the tool

    Returns:
        str: A generated SQL query.
    """
    # Read the Parquet file
    df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
    
    # Extract column names
    table_name = "sales"
    columns = ", ".join(df.columns)
    
    # Format the SQL generation prompt
    formatted_prompt = SQL_GENERATION_PROMPT.format(
        prompt=prompt, 
        columns=columns, 
        table_name=table_name
    )
    
    # Integrate with an LLM to generate the SQL query
    sql_query = query_agent.run(formatted_prompt)
    
    return sql_query

In [12]:
response = agent.run(test_prompt)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ generate sql query and execute to show all the sales for store 1320 on November 1st, 2021                       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-67e3f9d0-49cbdb30052d502751f34b16;17a08151-e551-4143-9f8f-5db7b4d52ed8)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 1.04 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-67e3f9d0-49cbdb30052d502751f34b16;17a08151-e551-4143-9f8f-5db7b4d52ed8)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [11]:
print(response)

SELECT Store_Number, SKU_Coded, Product_Class_Code, Sold_Date, Qty_Sold, Total_Sale_Value, On_Promo FROM sales WHERE Store_Number = 1320 AND Sold_Date = '2021-11-01';


### 2. Another tool for executing a query 

In [12]:
@tool
def execute_query_tool(query: str) -> str:
    """ 
    Executes a provided SQL query input is a query String.

    Args:
        query (str): The SQL query is string to be executed.

    Returns:
        str: The result of the SQL query execution as string.
    """
    # Step 1: Load parquet file into a pandas DataFrame.
    df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)

    # Step 2: Create the table in DuckDB if it doesn't already exist.
    duckdb.sql(f"CREATE TABLE IF NOT EXISTS sales AS SELECT * FROM df")

    # Step 3: Execute the provided SQL query.
    result = duckdb.sql(query).df()
    
    return result

In [24]:
# if not exist the table
# step 1: read the parquet file into a DuckDB table
df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

# execute the query 
result = duckdb.sql(response).df()
print(result)

NameError: name 'table_name' is not defined

In [11]:
from smolagents import HfApiModel, LiteLLMModel, OpenAIServerModel, TransformersModel, tool
from smolagents.agents import CodeAgent, ToolCallingAgent

model = LiteLLMModel(
        model_id="ollama_chat/qwen2.5-coder:1.5b",
        api_base="http://localhost:11434",  # replace with remote open-ai compatible server if necessary
        api_key="your-api-key",  # replace with API key if necessary
        num_ctx=8192,  # ollama default is 2048 which will often fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
    )

In [15]:
query_agent = CodeAgent(tools=[generate_sql_query,execute_query_tool],
                        model=model)
query_agent.run(test_prompt)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ generate sql query and execute to show all the sales for store 1320 on November 1st, 2021                       │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:1.5b ─────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  SELECT *                                                                                                         
  FROM sales                                                                                                       
  WHERE store_id = 1320 AND date = '2021-11-01';                                                                   
                                                                                                                   
  query = "SELECT * FROM sales WHERE store_id = 1320 AND date = '2021-11-01';"                                     
  result = execute_query_tool(query=query)                                                                         
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code parsing failed on line 1 due to: SyntaxError
SELECT *
         ^
Error: invalid syntax (<unknown>, line 1)

[Step 1: Duration 12.12 seconds| Input tokens: 2,162 | Output tokens: 124]

...

Code execution failed at line 'result = execute_query_tool(query=query)' due to: BinderException: Binder Error: 
Referenced column "store_id" not found in FROM clause!
Candidate bindings: "Store_Number", "Qty_Sold", "Product_Class_Code"

[Step 9: Duration 22.46 seconds| Input tokens: 33,020 | Output tokens: 2,379]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 10: Duration 28.51 seconds| Input tokens: 38,516 | Output tokens: 2,870]

KeyboardInterrupt: 

In [13]:
from smolagents import CodeAgent, tool, HfApiModel

# Tool to suggest a menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are only pass one value from given below:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

In [2]:
# Alfred, the butler, preparing the menu for the party
agent = ToolCallingAgent(tools=[suggest_menu], model=model, verbosity_level=2,max_steps = 3)

# Preparing the menu for the party
agent.run("Prepare a formal menu for the party.")

NameError: name 'ToolCallingAgent' is not defined

In [3]:
agent.prompt_templates

NameError: name 'agent' is not defined

In [ ]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-4319a85b-0552-42ce-aae1-2f7dd48a1668",
  public_key="pk-lf-73e457fc-6687-402e-aef5-c4448cc992f0",
  host="https://us.cloud.langfuse.com"
)